In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import glob
import keras
from functools import reduce
import xgboost
from xgboost import XGBClassifier

Using TensorFlow backend.
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tenso

In [2]:
frame1=pd.read_csv('/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/HR Features/HR Features_dataframe.csv')

In [3]:
frame2=pd.read_csv('/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Linear Accerelation Features/Linear Accerelation Features_dataframe.csv')

In [4]:
frame3=pd.read_csv('/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Angular Accerelation Features/Angular Accerelation Features_dataframe.csv')

In [5]:
result = pd.merge(frame1,frame2,on=["windowno","participant", 'ptsd_moment'])
frame= pd.merge(result,frame3,on=["windowno","participant", 'ptsd_moment'])

In [6]:
frame=frame.dropna()

In [7]:
frame.isnull().sum()

windowno          0
hrmean            0
hrmin             0
hrmax             0
hrsd              0
ptsd_moment       0
participant       0
hrrange           0
linaccmean        0
linaccmin         0
linaccmax         0
linaccsd          0
linear_accel_x    0
linear_accel_y    0
linear_accel_z    0
linaccrange       0
acc_x             0
acc_y             0
acc_z             0
acc_xmin          0
acc_ymin          0
acc_zmin          0
acc_xmax          0
acc_ymax          0
acc_zmax          0
acc_xsd           0
acc_ysd           0
acc_zsd           0
acc_xrange        0
acc_yrange        0
acc_zrange        0
dtype: int64

In [8]:
#X= frame[['linaccmean','linaccmin','linaccmax','linaccsd', 'linear_accel_x',
 #         'linear_accel_y','linear_accel_z','linaccrange',
  #        'hrmax', 'hrmean','hrmin','hrrange','hrsd']]



X= frame[['hrmax', 'hrmean','hrmin','hrrange','hrsd']]
#X= frameunder[['acc_x','acc_y','acc_z','linear_accel_x','linear_accel_y','linear_accel_z','linacc','hrmax', 'hrmean','hrmin','hrrange','hrsd']]
y= frame[['ptsd_moment']]

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [10]:
df1 = pd.DataFrame(X_train)

In [11]:
df2 = pd.DataFrame(y_train)

In [12]:
df = pd.concat([df1, df2], axis=1, sort=True)

In [13]:
frame0 = df[df['ptsd_moment'] ==0]
frame1 = df[df['ptsd_moment'] == 1]

In [14]:
count_class_0, count_class_1 = df.ptsd_moment.value_counts()

In [15]:
count_class_0

8958

In [16]:
ratio=0.75*count_class_0

In [17]:
frame_class_1_over = frame1.sample(int(ratio), replace=True)

In [18]:
frameover = pd.concat([frame0, frame_class_1_over], axis=0) 

In [19]:
##save training dataset

#frameover.to_csv (r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Training and Testing Data Sets/Training Data Set (without acc).csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [20]:
testingdataset = pd.concat([X_test,y_test], axis=1)

In [21]:
testingdataset.count()

hrmax          3993
hrmean         3993
hrmin          3993
hrrange        3993
hrsd           3993
ptsd_moment    3993
dtype: int64

In [22]:
#Save testing dataset

#testingdataset.to_csv (r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Training and Testing Data Sets/Testing Data Set (without acc).csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [23]:
#X_trainups= frameover[['acc_x','acc_y','acc_z','acc_xmin','acc_ymin',
 #         'acc_zmin','acc_xmax','acc_ymax','acc_zmax','acc_xsd',
  #        'acc_ysd','acc_zsd','acc_xrange','acc_yrange','acc_zrange', 'linaccmean','linaccmin','linaccmax','linaccsd', 'linear_accel_x',
   #       'linear_accel_y','linear_accel_z','linaccrange',
    #      'hrmax', 'hrmean','hrmin','hrrange','hrsd']]

    
X_trainups= frameover[[ 'hrmax', 'hrmean','hrmin','hrrange','hrsd']]    
#X= frameunder[['acc_x','acc_y','acc_z','linear_accel_x','linear_accel_y','linear_accel_z','linacc','hrmax', 'hrmean','hrmin','hrrange','hrsd']]
y_trainups= frameover[['ptsd_moment']]

In [24]:
#SOLVE XGBOOST NAME MISMATCH

X_trainupsarray = X_trainups.to_numpy()
y_trainupsarray= y_trainups.to_numpy()

In [25]:
y_trainupsarray.dtype

dtype('float64')

In [26]:
clf= XGBClassifier ()
#clf= RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=10)

In [27]:
clf.fit(X_trainups,y_trainups)

//miniconda3/envs/PTSD/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
//miniconda3/envs/PTSD/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [28]:
#For the watch and putting model on the watch
import coremltools



In [29]:
coreml_model = coremltools.converters.xgboost.convert(clf._Booster)

In [30]:
coreml_model.save('my_modelhrforwatch.mlmodel')

In [ ]:
from sklearn import metrics


predictions=clf.predict(X_test)

print(metrics.confusion_matrix(y_test,predictions))

In [ ]:
from sklearn.metrics import auc

fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions, pos_label=2)

In [ ]:
y_preds = clf.predict_proba(X_test)

# take the second column because the classifier outputs scores for
# the 0 class as well
preds = y_preds[:,1]
import matplotlib.pyplot as plt

In [ ]:
# fpr means false-positive-rate
# tpr means true-positive-rate
fpr, tpr, _ = metrics.roc_curve(y_test, preds)

auc_score = metrics.auc(fpr, tpr)

# clear current figure
plt.clf()

plt.title('ROC Curve')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

# it's helpful to add a diagonal to indicate where chance 
# scores lie (i.e. just flipping a coin)
plt.plot([0,1],[0,1],'r--')

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc='lower right')
#It either does plot show ot plot save if you wanna save the plot make sure you don't execute plt show
#plt.show()
plt.savefig('Xgboost.png', dpi=300)

In [ ]:
# plot feature importance
from xgboost import plot_importance
plot_importance(clf)
#plt.show()
plt.savefig('FeatureImportanceXgboost.png', dpi=200)

In [ ]:
#Feature importance xgoost and xgboostregressor
fi = pd.DataFrame({'feature': list(X_trainups.columns),
                   'importance': clf.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

In [ ]:
Confusion = pd.DataFrame(metrics.confusion_matrix(y_test,predictions), 
                         index=['NO PTSD','PTSD'], 
                         columns=['NO PTSD','PTSD'])
Confusion

accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
Confusion

In [ ]:
print(metrics.classification_report(y_test,predictions))

